# Algoritma *Naive Bayes*

## Mari berkenalan dengan Algoritma Naive Bayes

insert penjelasan here...

## Mengimpor Pustaka

Pada bagian ini akan diimpor beberapa pustaka yang akan digunakan untuk menyimulasikan Algoritma *K-Nearest Neighbor*.

In [1]:
import pandas as pd
import numpy as np

Selain itu, akan dipanggil juga modul KNN yang telah dibangun *from scratch*

In [2]:
from algorithm.naiveBayes import NaiveBayes

## Mengimpor *Dataset*

Pada bagian ini akan diimpor *dataset* yang sebelumnya telah terbagi menjadi `data_train.csv` dan `data_validation.csv`.

In [3]:
# Mengambil data train dan data validation
df_train = pd.read_csv("../data/data_train.csv")
df_validation = pd.read_csv("../data/data_validation.csv")

# *Pre-processing* Data

Tahap yang dilakukan meliputi pemisahan kolom target hingga melakukan standarisasi terhadap data sebelum dilakukan pemrosesan dengan Algoritma KNN.

In [4]:
# Melakukan pemisahan kolom target
x_train = df_train.drop(["price_range"], axis=1)
y_train = df_train["price_range"]

x_test = df_validation.drop(["price_range"], axis=1)
y_test = df_validation["price_range"]

In [5]:
# Standarisasi data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Hasil Pemrosesan Algoritma Naive Bayes yang Dibangun

Berikut adalah hasil pemrosesan Algoritma Naive Bayes yang dibangun *from scratch*.

In [6]:
# Gunakan model KNN yang sebelumnya dibangun
nb_scratch = NaiveBayes() 

# Lakukan fit model
nb_scratch.fit(x_train, y_train)

# Lakukan prediksi dengan data validation
y_pred_scratch = nb_scratch.predict(x_test)

In [7]:
# Pengujian kualitas model dengan metrik
from sklearn.metrics import accuracy_score, classification_report

print(classification_report(y_test, y_pred_scratch))
print("Akurasi : ", 100 * np.round(accuracy_score(y_test, y_pred_scratch), 5), "%")

              precision    recall  f1-score   support

           0       0.88      0.91      0.89       142
           1       0.68      0.69      0.68       144
           2       0.69      0.70      0.69       155
           3       0.92      0.86      0.89       159

    accuracy                           0.79       600
   macro avg       0.79      0.79      0.79       600
weighted avg       0.79      0.79      0.79       600

Akurasi :  79.0 %


## Hasil Pemrosesan Algoritma Naive Bayes Pembanding

Hasil pemrosesan diatas akan dibandingkan dengan hasil yang diperoleh dari *library* scikit-learn

In [8]:
# Pemanggilan model KNN dari scikit-learn
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train)
y_pred_scikit = gnb.predict(x_test)

In [9]:
# Pengujian kualitas model dengan metrik
print(classification_report(y_test, y_pred_scikit))
print("Akurasi : ", 100 * np.round(accuracy_score(y_test, y_pred_scikit), 5), "%")

              precision    recall  f1-score   support

           0       0.87      0.88      0.88       142
           1       0.66      0.65      0.65       144
           2       0.68      0.71      0.70       155
           3       0.90      0.89      0.90       159

    accuracy                           0.78       600
   macro avg       0.78      0.78      0.78       600
weighted avg       0.78      0.78      0.78       600

Akurasi :  78.167 %


## Hasil yang diperoleh

Berdasarkan kedua hasil tersebut, diperoleh **hasil yang sama** antara nilai prediksi yang dihasilkan oleh model Naive Bayes yang dibangun *from scratch* dengan model yang dimiliki scikit-learn.

In [10]:
# Pengujian kualitas model dengan metrik
print(classification_report(y_pred_scratch, y_pred_scikit))

              precision    recall  f1-score   support

           0       0.98      0.95      0.97       147
           1       0.95      0.90      0.92       148
           2       0.91      0.94      0.93       156
           3       0.94      0.99      0.96       149

    accuracy                           0.94       600
   macro avg       0.95      0.94      0.95       600
weighted avg       0.95      0.94      0.94       600



## Apakah sudah merupakan model yang terbaik?

OTW diubah

## Penyimpanan dan *Load* Model

Agar model dapat digunakan kembali, maka model harus dapat disimpan dan di-*load*. Berikut adalah implementasi penyimpanan model yang dilakukan menggunakan *library* pickle.

In [11]:
import pickle

# Menyimpan model dalam pkl
model_pkl_file = "models/naive_bayes_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(nb_scratch, file)

Untuk membuktikan bahwa model berhasil tersimpan, berikut adalah pembuktian pemanggilan kembali hasil prediksi model yang disimpan

In [12]:
# Load kembali model dari pkl
with open(model_pkl_file, 'rb') as file:  
    model = pickle.load(file)

# Melakukan prediksi dengan model tersebut
y_pickle = model.predict(x_test)

# Menguji hasil akurasi model
print(classification_report(y_test, y_pickle)) 

              precision    recall  f1-score   support

           0       0.88      0.91      0.89       142
           1       0.68      0.69      0.68       144
           2       0.69      0.70      0.69       155
           3       0.92      0.86      0.89       159

    accuracy                           0.79       600
   macro avg       0.79      0.79      0.79       600
weighted avg       0.79      0.79      0.79       600



Terbukti bahwa model pkl yang disimpan berhasil untuk digunakan kembali.

## [Bonus] Submisi Kaggle

Bagian ini dikhususkan untuk pemrosesan data dan penggunaan model yang dibuat sebagai dasar membuat submisi pada Kaggle.

In [13]:
# Impor dataset
test_data = pd.read_csv('../data/test.csv')

features = test_data.drop(["id"], axis=1)  # menghapus kolom "id"

# Gunakan model KNN yang sebelumnya dibangun
nb_kaggle = NaiveBayes()

# Lakukan fit model
nb_kaggle.fit(x_train, y_train)

# Lakukan prediksi dengan data validation
feature_test = scaler.transform(features)
predictions = nb_kaggle.predict(feature_test)

# Membuat dataframe hasil
result_df = pd.DataFrame({'id': test_data['id'], 'price_range': predictions})

# Menyimpan dataframe dalam csv
result_df.to_csv('../result/predictions-naive-bayes.csv', index=False)